In [127]:
import itertools
import re
import random
import numpy as np
import math
from timeit import default_timer as timer
from pysuffixarray.core import SuffixArray
from collections import defaultdict
import RMQ as rmq
import regex

In [3]:
def load_text(file='../train data/P4b/sentences_for_task1.txt'):
    complete_text = ''
    with open(file, 'r', encoding='utf8') as base_vectors_lines:
        for line in base_vectors_lines:
            line = line.strip().lower()
            complete_text += ' ' + line

    complete_text = complete_text + ' '
    return complete_text

text = load_text()
sabase = SuffixArray(text)
SA = sabase.suffix_array()
LCP = sabase.longest_common_prefix()

In [207]:
precomp_length = 16
precomp_adj = 4
alphabet = [' ', 'a', 'ą', 'b', 'c', 'ć', 'd', 'e', 'ę', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'ł', 'm', 'n','ń', 'o', 'ó', 'p', 'q', 'r', 's', 'ś', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ź', 'ż']

def precompute_counts(ln):
    word_counts = {}
    template = {}
    for a in alphabet:
        template[a] = 0
    adj_table = {}
    for i in range(len(SA)):
        for j in range(1, ln):
            wrd = text[SA[i]:SA[i]+j]
            if wrd not in word_counts:
                word_counts[wrd] = [0, i]
                if j <= precomp_adj:
                    adj_table[wrd] = [template.copy(), template.copy()]
            word_counts[wrd][0] += 1
            if j <= precomp_adj:
                if SA[i]-1 > 0:
                    adj_table[wrd][0][text[SA[i]-1]] += 1
                if SA[i]+j < len(text):
                    adj_table[wrd][1][text[SA[i]+j]] += 1

    return word_counts, adj_table

In [11]:
word_counts, adj_table = precompute_counts(precomp_length)

In [26]:
def count_letters():
    letters = defaultdict(int)
    for l in text: 
        letters[l] += 1
    return letters, sum([v for v in letters.values()])
letters, letters_sum = count_letters()

rmq = rmq.RMQ(len(SA))
for i in range(len(LCP)):
    rmq.update(i, LCP[i])

In [49]:
def check_prefix(pre, w, init=0):
    ln = len(pre)
    lw = len(w)
    for i in range(init, ln, 1):
        if lw <= i:
            return False, True
        if w[i] != pre[i]:
            return i, w[i] < pre[i]
    return ln, True

def binsearch_h(w, li, ri):
    i = li
    while li < ri:
        preli = li
        preri = ri
        i = (li+ri)//2
        lw = len(w)
        ans = rmq.query(li+1, i+1)
        if ans < lw:
            ri = i
        else:
            li = i
        if preli == li and preri == ri:
            return li
    return li

def binsearch_l(w, li, ri):
    i = ri
    while li < ri:
        preli = li
        preri = ri
        i = (li+ri)//2
        lw = len(w)
        ans = rmq.query(i+1, ri+1)
        if ans < lw:
            li = i
        else:
            ri = i
        if preli == li and preri == ri:
            return ri
    return ri

def count_words(w):
    if len(w) <= precomp_length:
        if w in word_counts:
            return word_counts[w]
        return 0, 10
    li = 0
    ri = len(SA)-1
    lw = len(w)
    i = -1
    ans = 0
    while li < ri:
        preli = li
        preri = ri
        prei = i
        i = (li+ri)//2
        if prei < i:
            init = rmq.query(prei+1, i+1)
        else:
            init = rmq.query(i+1, prei+1)
        init = min(init, ans)
        ans, direction = check_prefix(w, text[SA[i]:], init)
        if ans < lw:
            if direction:
                li = i
            else:
                ri = i
            if preli == li and preri == ri:
                return 0, 0
        else:
            h = binsearch_h(w, i, ri)
            l = binsearch_l(w, li, i)
            return h - l + 1, l


In [27]:
def check_adj(w):
    if len(w) <= precomp_adj:
        return adj_table[w]
    bs, sp = count_words(w)
    prefixes = defaultdict(int)
    suffixes = defaultdict(int)
    if bs > 1000000:
        for a in alphabet:
            pre, _ = count_words(a + w)
            suf, _ = count_words(w + a)
            prefixes[a] = pre
            suffixes[a] = suf
    else:
        for i in range(sp, sp+bs):
            if SA[i] - 1 > 0:
                prefixes[text[SA[i]-1]] += 1
            if SA[i] + len(w) < len(text):
                suffixes[text[SA[i]+len(w)]] += 1
    return prefixes, suffixes

In [204]:
def suffix_prefix_heu(counter):
    sm = max(1, sum(counter.values()))
    mx = max(counter.values())
    res = 0
    for pp1, pp2 in [(letters[a]/letters_sum, counter[a]/sm) for a in alphabet]:
        res += (pp2-pp1)*(pp2-pp1)
    return res if counter[' '] != 0 and mx <= 3 else 0.8

In [173]:
def check_pair(w1, w2):
    ww1, ww2 = check_adj(w1), check_adj(w2)
    ans1 = (suffix_prefix_heu(ww1[0]) ** 2 + suffix_prefix_heu(ww1[1]) ** 2)
    ans2 = (suffix_prefix_heu(ww2[0]) ** 2 + suffix_prefix_heu(ww2[1]) ** 2)
    if ans1 > ans2:
        return -1
    elif ans1 < ans2:
        return 1
    return 0

In [104]:
def load_tests(file='../train data/P4b/test_for_task1.txt'):
    complete_database = []
    i = 0
    with open(file, 'r', encoding='utf8') as base_vectors_lines:
        for line in base_vectors_lines:
            line = line.strip()
            line = line.split(' ')
            complete_database.append(line[0])
            complete_database.append(line[1])
            i += 1
    return complete_database
tests = load_tests()

In [105]:
def resolve_tests(tests, debug=False):
    score = 0
    for i in range(0, len(tests), 2):
        res = check_pair(tests[i], tests[i+1])
        score += 1 if res == 1 else (0.5 if res == 0 else 0)
        if debug:
            print(tests[i], tests[i+1], res)
    return (score*2) / len(tests)

In [215]:
resolve_tests(tests)

0.851315

## Zad 2

In [65]:
text2 = load_text('../train data/P4b/sentences_for_task2.txt')
text_str = ''.join(text2)
tests2 = load_tests('../train data/P4b/test_for_task2.txt')

In [206]:
def run_tests2(tests, debug=False):
    score = 0
    for i in range(0, len(tests), 2):
        t1 = len(regex.findall("(" + tests[i]   + ")", text_str))
        t2 = len(regex.findall("(" + tests[i+1] + ")", text_str))
        if t1 > t2:
            score += 1
        elif t1 == t2:
            score += 0.5
        if debug:
            print(tests[i], tests[i+1], 1 if t1 > t2 else 0 if t1 == t2 else -1)
    return (score*2) / len(tests)

In [228]:
run_tests2(tests2, debug=False)

0.816825

## zad1a

#### bez zmian:

In [5]:
!swipl -c np_original.pl | grep -c GOOD

% Disabled autoloading (loaded 28 files)
% Disabled autoloading (loaded 2 files)
% Disabled autoloading (loaded 0 files)
6106


#### po zmianach:

In [6]:
!swipl -c np_original.pl | grep -c GOOD

% Disabled autoloading (loaded 28 files)
% Disabled autoloading (loaded 2 files)
% Disabled autoloading (loaded 0 files)
8051
